The `freeze_to` source code can be understood as the following pseudo-code:
```python
def freeze_to(self, n:int)->None:
    for g in self.layer_groups[:n]: freeze 
    for g in self.layer_groups[n:]: unfreeze
```    
In other words, for example, `freeze_to(1)` is to freeze layer group 0 and unfreeze the rest layer groups, and `freeze_to(3)` is to freeze layer groups 0, 1, and 2 but unfreeze the rest layer groups (if there are more layer groups left).       

Both `freeze` and `unfreeze` [sources](https://github.com/fastai/fastai/blob/master/fastai/basic_train.py#L216) are defined using `freeze_to`:    
- When we say `freeze`, we mean that in the specified layer groups the `requires_grad` of all layers with weights (except BatchNorm layers) are set `False`, so the layer weights won't be updated during training.     
- when we say `unfreeze`, we mean that in the specified layer groups the `requires_grad` of all layers with weights (except BatchNorm layers) are set `True`, so the layer weights will be updated during training.

You can experiment `freeze_to`, `freeze` and `unfreeze` with the following experiment.

In [ ]:
import fastai.vision as fv

In [ ]:
fv.__version__

In [ ]:
path_test =  fv.Path('/kaggle/input/test');
path_train = fv.Path('/kaggle/input/train'); path_train.ls()

In [ ]:
fv.np.random.seed(1)

### 创建DataBunch

data = fv.ImageDataBunch.from_folder(path_train,
                                  test=path_test, 
                                  ds_tfms=fv.get_transforms(),
                                  valid_pct=0.25,
                                  size=128, 
                                  bs=32,
                                  num_workers=0)
data.normalize(fv.imagenet_stats)
data

In [ ]:
learn = fv.cnn_learner(data, 
                      fv.models.resnet18, 
                      metrics=fv.error_rate,
                      model_dir="/kaggle/working/")

In [ ]:
learn.save('start')
!ls .

```python
learn.freeze_to??


Signature: learn.freeze_to(n:int) -> None
Source:   
    def freeze_to(self, n:int)->None:
        "Freeze layers up to layer group `n`."
        for g in self.layer_groups[:n]:
            for l in g:
                if not self.train_bn or not isinstance(l, bn_types): requires_grad(l, False)
        for g in self.layer_groups[n:]: requires_grad(g, True)
        self.create_opt(defaults.lr)
File:      /opt/conda/lib/python3.6/site-packages/fastai/basic_train.py
Type:      method
```

#### How many layer_groups in Resnet 18

In [ ]:
print('there are ', len(learn.layer_groups), 'layer_groups in this leaner object')

In [ ]:

for g in learn.layer_groups[:]:
    print(len(g), 'layers')
    num_trainables = fv.np.array([hasattr(l, 'weight') for l in g]).sum()
    print(num_trainables, 'layers with weights')
    num_bn = fv.np.array([isinstance(l, fv.bn_types) for l in g]).sum()
    print(num_bn, "BN layers Not be frozen")
    print(num_trainables - num_bn, 'layers which can be frozen')
    print('')
    print(g)

In [ ]:
learn.summary()

In [ ]:
l = learn.layer_groups[0][0]; l

In [ ]:
learn.train_bn

In [ ]:
print(fv.bn_types)
isinstance(l, fv.bn_types)

fv.requires_grad?

```python
Signature: fv.requires_grad(m:torch.nn.modules.module.Module, b:Union[bool, NoneType]=None) -> Union[bool, NoneType]
Docstring: If `b` is not set return `requires_grad` of first param, else set `requires_grad` on all params as `b`
File:      /opt/conda/lib/python3.6/site-packages/fastai/torch_core.py
Type:      function
```

In [ ]:
fv.requires_grad(l, False)

In [ ]:
learn.freeze_to(0) # freeze layer group before group 0
learn.summary()

In [ ]:
learn.freeze_to(1) # freeze layer group before group 1
learn.summary()

```python
learn.freeze??

Signature: learn.freeze() -> None
Source:   
    def freeze(self)->None:
        "Freeze up to last layer group."
        assert(len(self.layer_groups)>1)
        self.freeze_to(-1)
        self.create_opt(defaults.lr) # also create an optimizer for learner
File:      /opt/conda/lib/python3.6/site-packages/fastai/basic_train.py
Type:      method
```

In [ ]:
len(learn.layer_groups)

In [ ]:
assert(len([1,2])>1)
# assert(len([2])>1)

In [ ]:
learn.create_opt?

```python
learn.unfreeze??

Signature: learn.unfreeze()
Source:   
    def unfreeze(self):
        "Unfreeze entire model."
        self.freeze_to(0)
        self.create_opt(defaults.lr) # then create an optimizer for learner
File:      /opt/conda/lib/python3.6/site-packages/fastai/basic_train.py
Type:      method
```